## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
f.predictions('nvda')

NVDA 5m Interval Timestamp: 2024-10-16 03:28:03 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,static,static,down
kelly_1:2.5,0.3,0.640166,0.353067,0.292318
prob_up,0.25,0.001833,0.018809,0.028864
prob_static,0.25,0.742975,0.537905,0.476624
prob_down,0.5,0.255191,0.443286,0.494513
precision,0.770791,0.731707,0.689716,0.683292
recall,0.769231,0.787879,0.785859,0.554656
f1,0.77001,0.758755,0.734655,0.612291
support,"[494.0, 495.0, 494.0]","[494.0, 495.0, 494.0]","[494.0, 495.0, 494.0]","[494.0, 495.0, 494.0]"


NVDA 15m Interval Timestamp: 2024-10-16 03:28:12 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,static,static,static
kelly_1:2.5,0.3,0.661161,0.880631,0.421783
prob_up,0.25,0.002477,0.005254,0.09233
prob_static,0.25,0.757972,0.914737,0.586988
prob_down,0.5,0.239551,0.080009,0.320682
precision,0.813433,0.706806,0.706468,0.657609
recall,0.668712,0.833333,0.876543,0.746914
f1,0.734007,0.764873,0.782369,0.699422
support,"[162.0, 162.0, 163.0]","[162.0, 162.0, 163.0]","[162.0, 162.0, 163.0]","[162.0, 162.0, 163.0]"


NVDA 30m Interval Timestamp: 2024-10-16 03:28:15 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,down,down
kelly_1:2.5,0.125,0.94872,0.939026,0.76516
prob_up,0.25,0.003914,0.011972,0.081023
prob_static,0.375,0.032715,0.03158,0.08672
prob_down,0.375,0.963371,0.956447,0.832257
precision,0.707317,0.772201,0.773585,0.736607
recall,0.696,0.809717,0.82996,0.668016
f1,0.701613,0.790514,0.800781,0.700637
support,"[246.0, 250.0, 247.0]","[246.0, 250.0, 247.0]","[246.0, 250.0, 247.0]","[246.0, 250.0, 247.0]"


NVDA 1wk Interval Timestamp: 2024-10-16 03:28:32 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,static,up
kelly_1:2.5,0.3,0.373368,0.233048,0.158835
prob_up,0.5,0.552406,0.394221,0.399168
prob_static,0.25,0.327663,0.452177,0.351638
prob_down,0.25,0.119931,0.153602,0.249194
precision,0.744681,0.73913,0.809524,0.645833
recall,0.729167,0.708333,0.68,0.645833
f1,0.736842,0.723404,0.73913,0.645833
support,"[48.0, 50.0, 48.0]","[48.0, 50.0, 48.0]","[48.0, 50.0, 48.0]","[48.0, 50.0, 48.0]"


NVDA 1mo Interval Timestamp: 2024-10-16 03:28:35 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,static,static
kelly_1:2.5,0.3,0.338174,0.602417,0.322032
prob_up,0.5,0.527267,0.198285,0.357033
prob_static,0.375,0.467402,0.716012,0.515737
prob_down,0.125,0.005331,0.085702,0.12723
precision,0.857143,0.785714,0.823529,0.8125
recall,0.8,0.733333,0.777778,0.722222
f1,0.827586,0.758621,0.8,0.764706
support,"[15.0, 18.0, 15.0]","[15.0, 18.0, 15.0]","[15.0, 18.0, 15.0]","[15.0, 18.0, 15.0]"


### Check Summary Output

In [ ]:
symbol='NVDA'
interval='5m'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')